In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import pandas as pd
import numpy as np

In [3]:
processed = 'processed'
submissions = 'submissions'
features = 'features'
df_train = pd.read_csv(f'{processed}/train.csv')
df_test =  pd.read_csv(f'{processed}/test.csv')

In [4]:
!ls {features}

age_human_test.csv        age_quantiles_train.csv   name_features_test.csv
age_human_train.csv       family_features_test.csv  name_features_train.csv
age_q_test.csv            family_features_train.csv pclass_dummy_test.csv
age_q_train.csv           fare_quantiles_test.csv   pclass_dummy_train.csv
age_quantiles_test.csv    fare_quantiles_train.csv


In [5]:
for name in ['age_human', 'age_quantiles', 'family_features', 'fare_quantiles', 'name_features', 'pclass_dummy']:
    df_train = df_train.merge(pd.read_csv(f'{features}/{name}_train.csv'))
    df_test =  df_test.merge(pd.read_csv(f'{features}/{name}_test.csv'))

In [6]:
feature_names = [name for name in df_test.columns if name not in [
    'Name', 'src', 'Sex', 'Cabin', 'Embarked', 'Ticket', 'Fare', 'Survived', 'Name_last', 'Age']]
feature_names

['PassengerId',
 'SibSp',
 'Parch',
 'Sex_female',
 'Sex_male',
 'Fare_is_na',
 'Fare_q_is(-0.001..7.568]',
 'Fare_q_is(7.568..7.854]',
 'Fare_q_is(7.854..8.05]',
 'Fare_q_is(8.05..10.5]',
 'Fare_q_is(10.5..14.454]',
 'Fare_q_is(14.454..21.679]',
 'Fare_q_is(21.679..26.955]',
 'Fare_q_is(26.955..41.579]',
 'Fare_q_is(41.579..78.051]',
 'Fare_q_is(78.051..512.329]',
 'Age_na',
 'Age_is(0..2]',
 'Age_is(2..6]',
 'Age_is(6..14]',
 'Age_is(14..16]',
 'Age_is(16..18]',
 'Age_is(18..25]',
 'Age_is(25..35]',
 'Age_is(35..40]',
 'Age_is(40..45]',
 'Age_is(45..60]',
 'Age_is(60..80]',
 'Age_q_is(0.169..14.0]',
 'Age_q_is(14.0..19.0]',
 'Age_q_is(19.0..22.0]',
 'Age_q_is(22.0..25.0]',
 'Age_q_is(25.0..28.0]',
 'Age_q_is(28.0..31.0]',
 'Age_q_is(31.0..36.0]',
 'Age_q_is(36.0..42.0]',
 'Age_q_is(42.0..50.0]',
 'Age_q_is(50.0..80.0]',
 'Pclass_is_1',
 'Pclass_is_2',
 'Pclass_is_3',
 'Name_title_survival_odds',
 'Name_title_is_Master.',
 'Name_title_is_Miss.',
 'Name_title_is_Mr.',
 'Name_title_is_M

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

target_column = 'Survived'
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(solver='lbfgs', max_iter=1000), n_features_to_select=5, step=1)
rfe_selector.fit(df_train[feature_names], df_train[target_column])

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=1000,
                                 multi_class='warn', n_jobs=None, penalty='l2',
                                 random_state=None, solver='lbfgs', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=5, step=1, verbose=0)

In [8]:
rfe_support = rfe_selector.get_support()
rfe_feature = df_train[feature_names].loc[:,rfe_support].columns.tolist()
rfe_feature

['Sex_female',
 'Name_title_is_Master.',
 'Name_title_is_Mr.',
 'Name_title_is_Other.',
 'Name_last_odds_survival']

In [9]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2", solver='lbfgs', max_iter=1000), max_features=5)
embeded_lr_selector.fit(df_train[feature_names], df_train[target_column])

SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None,
                                             dual=False, fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=1000, multi_class='warn',
                                             n_jobs=None, penalty='l2',
                                             random_state=None, solver='lbfgs',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=5, norm_order=1, prefit=False, threshold=None)

In [10]:
embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = df_train[feature_names].loc[:,embeded_lr_support].columns.tolist()
embeded_lr_feature

['Name_title_is_Master.',
 'Name_title_is_Mr.',
 'Name_title_is_Mrs.',
 'Name_title_is_Other.',
 'Name_last_odds_survival']

In [11]:
# Note to self: I bet having a reliable cross-validation strategy would have helped.
# Note to self: all the title_* features selected here automatically remind a thing I've read on title-based model by a human which scored well (imperatively constructed though)